In [1]:
import io

In [2]:
prefix_file = 'prefix.txt'
suffix_file = 'suffix.txt'

interval = 5

script = []
G_set = {}

with open('human_script.txt') as input:
    for line in input.readlines():
        if line != '' and line != '\n':
            line = line.rstrip()
            
            if '//' in line:
                #ignore comments
                continue
            
            elif 'group' in line:
                G = line.split(' ')[1]
                print(G)
                #for each group, make a new dict in G_set dict
                G_set[G] = {}
                id_list = line.partition('=')[2]
                id_list = ''.join(id_list.split())
                id_list = id_list.split(',')
                G_set[G]['ids'] = id_list
                G_set[G]['last_height'] = 0
                print(G_set[G])
                
            elif ':' in line:
                G = line.partition(':')[0]
                print(G)
                time = line.split(' ')[1].rstrip('s')
                command = line.split(' ')[2]
    
                if 'takeoff' in command:
                    G_set[G][int(time)] = 'takeoff'
                    #assume takeoff height is 50....
                    G_set[G]['last_height'] = 50
                    print('added takeoff command at %ss' % time)
                    
                elif '-' in time:
                    start = int(time.partition('-')[0].rstrip('s'))
                    end = int(time.partition('-')[2].rstrip('s'))
                    duration = end - start
                    distance = int(command) - G_set[G]['last_height']
                    
                    speed = abs(distance / duration)
                    
                    print('group %s moving from %i to %i' % (G, G_set[G]['last_height'], int(command)))
                    print('move duration: %i' % duration)
                    print('move distance: %i' % distance)
                    print('move speed: %f' % speed)
                    
                    intervals = duration // interval
                    inter_dist = distance / intervals
                    print('move occurs over %f intervals, %f per interval' % (intervals, inter_dist))
                    
                    for i in range(int(intervals)):
                        timestamp = start + (i * interval)
                        G_set[G][timestamp] = 'go 0 0 %f %f' % (inter_dist, speed)
                        
                    
                    G_set[G]['last_height'] = int(command)
        
    input.close()
    
for key in G_set.keys():
    print('\ngroup %s final dict:' % key)
    print(G_set[key])

A
{'ids': ['9', '10', '11', '12', '13', '14', '15'], 'last_height': 0}
B
{'ids': ['16', '17', '4', '19', '20'], 'last_height': 0}
C
{'ids': ['21', '22', '23', '24'], 'last_height': 0}
D
{'ids': ['25', '26', '27'], 'last_height': 0}
E
{'ids': ['28', '29'], 'last_height': 0}
A
added takeoff command at 0s
A
group A moving from 50 to 250
move duration: 20
move distance: 200
move speed: 10.000000
move occurs over 4.000000 intervals, 50.000000 per interval
B
added takeoff command at 20s
B
group B moving from 50 to 225
move duration: 10
move distance: 175
move speed: 17.500000
move occurs over 2.000000 intervals, 87.500000 per interval
C
added takeoff command at 40s
C
group C moving from 50 to 200
move duration: 15
move distance: 150
move speed: 10.000000
move occurs over 3.000000 intervals, 50.000000 per interval
D
added takeoff command at 60s
D
group D moving from 50 to 150
move duration: 15
move distance: 100
move speed: 6.666667
move occurs over 3.000000 intervals, 33.333333 per interval


In [3]:
with open(prefix_file, 'r') as prefix:
    for line in prefix.readlines():
        script.append(line)
    prefix.close()

In [4]:
full_duration = 100
inter_steps = range(0, full_duration, 5)

for step in inter_steps:
    script.append('//time(seconds) = %s\n' % step)
    for G in G_set.keys():
        if step in G_set[G].keys():
            for _id in G_set[G]['ids']:
                this_line = _id + '>' + G_set[G][step] +'\n'
                script.append(this_line)
        else:
            for _id in G_set[G]['ids']:
                this_line = _id + '>stop\n'
                script.append(this_line)
    script.append('sync %i\n' % interval)

In [5]:
with open(suffix_file, 'r') as suffix:
    for line in suffix.readlines():
        script.append(line)
    suffix.close()

In [6]:
with open('..\\machine_script.txt', 'w') as output:
    output.writelines(script)
    output.close